In [1]:
from __future__ import print_function, division
from keras.constraints import max_norm
from keras.regularizers import l2
from custom_layers import Conv1D_linearphase
from heartnet_v1 import heartnet, reshape_folds, branch
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
set_session(tf.Session(config=config))
import numpy as np
np.random.seed(1)
from tensorflow import set_random_seed
set_random_seed(1)
from keras.utils import to_categorical
from keras.optimizers import Adam, SGD
from keras.callbacks import CSVLogger, ModelCheckpoint, TensorBoard, Callback
from keras.utils import plot_model
from keras.layers import Dense, Dropout, Concatenate, initializers, Input
from keras.models import Model
from keras import backend as K
from sklearn.metrics import recall_score, confusion_matrix
import pandas as pd
import os
import tables
from datetime import datetime
from hyperopt import hp, fmin, Trials, tpe, STATUS_OK

Using TensorFlow backend.


In [2]:
def compute_weight(Y, classes):
    num_samples = (len(Y))
    n_classes = (len(classes))
    num_bin = np.sum(Y,axis=-2)
    class_weights = {i: (num_samples / (n_classes * num_bin[i])) for i in range(n_classes)}
    return class_weights

In [3]:
def heartnet_transfer(load_path='/home/prio/heart_sound/weights.0148-0.8902.hdf5',
                      lr=0.0012843784,lr_decay=0.0001132885,
                      num_dense1=20,num_dense2=20,trainable=False,dropout_rate=0.):
    model = heartnet(load_path=False,FIR_train=False,trainable=trainable)
    plot_model(model,'before.png',show_shapes=True,show_layer_names=True)
    x = model.layers[-4].output
    x = Dense(num_dense1,activation='relu',kernel_initializer=initializers.he_uniform(seed=1)) (x)
    x = Dropout(rate=dropout_rate,seed=1) (x)
    x = Dense(num_dense2, activation='relu',kernel_initializer=initializers.he_normal(seed=1))(x)
    x = Dropout(rate=dropout_rate, seed=1)(x)
    output = Dense(3,activation='softmax')(x)
    model = Model(inputs=model.input,outputs=output)
    plot_model(model, 'after.png',show_shapes=True,show_layer_names=True)
    if load_path:
        model.load_weights(load_path,by_name=True)
    sgd = SGD(lr=lr)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [4]:
class log_UAR(Callback):

    def __init__(self, x_val, y_val, val_parts):
        self.x_val = x_val
        self.y_val = y_val
        self.val_parts = val_parts


    def on_epoch_end(self, epoch, logs):
        if logs is not None:
            y_pred = self.model.predict(self.x_val, verbose=0)
            y_pred = np.argmax(y_pred, axis=-1)
            self.y_val_ = np.transpose(np.argmax(self.y_val, axis=-1))
            true = []
            pred = []
            start_idx = 0
            for s in self.val_parts:

                if not s:  ## for e00032 in validation0 there was no cardiac cycle
                    continue
                # ~ print "part {} start {} stop {}".format(s,start_idx,start_idx+int(s)-1)

                temp_ = self.y_val_[start_idx:start_idx + int(s) - 1]
                temp = y_pred[start_idx:start_idx + int(s) - 1]

                if (sum(temp == 0) > sum(temp == 1)) and (sum(temp == 0) > sum(temp == 2)):
                    pred.append(0)
                elif (sum(temp == 2) > sum(temp == 1)) and (sum(temp == 2) > sum(temp == 0)):
                    pred.append(2)
                else:
                    pred.append(1)

                if (sum(temp_ == 0) > sum(temp_ == 1)) and (sum(temp_ == 0) > sum(temp_ == 2)):
                    true.append(0)
                elif (sum(temp_ == 2) > sum(temp_ == 1)) and (sum(temp_ == 2) > sum(temp_ == 0)):
                    true.append(2)
                else:
                    true.append(1)

                start_idx = start_idx + int(s)

            score = recall_score(y_pred=pred, y_true=true, average='macro')
            logs['UAR'] = np.array(score)

In [5]:
fold_dir = '/home/prio/heart_sound/feature/segmented_noFIR/'
foldname = 'comParE'
model_dir = '/home/prio/heart_sound/models/'
log_dir = '/home/prio/heart_sound/logs/'

##### Load Model ######
load_path='/home/prio/heart_sound/weights.0148-0.8902.hdf5'
# lr = 0.00001
# lr = 0.1
# num_dense1 = 734 #34,120,167,239,1239,650,788,422,598
# num_dense2 = 20 #121,
epochs = 45
batch_size = 256
dropout_rate = 0.
trainable = True
addweights = True

In [6]:
feat = tables.open_file(fold_dir + foldname + '.mat')
x_train = feat.root.trainX[:]
y_train = feat.root.trainY[0, :]
x_val = feat.root.valX[:]
y_val = feat.root.valY[0, :]
train_parts = feat.root.train_parts[:]
val_parts = feat.root.val_parts[0, :]
############### Reshaping ############
x_train, y_train, x_val, y_val = reshape_folds(x_train, x_val, y_train, y_val)
y_train = to_categorical(y_train,num_classes=3)
y_val = to_categorical(y_val,num_classes=3)

(17061, 2500, 1)
(17061, 1)
(5872, 2500, 1)
(5872, 1)


In [7]:
def objective(args):
    #### Parse arguments and print #####
    
    print("args %s" % args)
    
    lr = args['lr']
    num_dense1 = args['num_dense1']
    num_dense2 = args['num_dense2']
    
    ##### Load model ######
    model = heartnet_transfer(load_path=load_path,lr=lr,num_dense1=num_dense1,
                              num_dense2=num_dense2,trainable=trainable,
                              dropout_rate=dropout_rate)
    
    #### Log params #####
    log_name = foldname + ' ' + str(datetime.now()) + str([lr,num_dense1,num_dense2])
    checkpoint_name = model_dir + log_name + "/" + 'weights.{epoch:04d}-{val_acc:.4f}.hdf5'
    if not os.path.exists(model_dir + log_name):
        os.makedirs(model_dir + log_name)
    plot_model(model,"model.png",show_layer_names=True,show_shapes=True)
    
    ### Callbacks ###
    
    csv_logger = CSVLogger(log_dir + log_name + '/training.csv')
    modelcheckpnt = ModelCheckpoint(filepath=checkpoint_name,
                                    monitor='val_acc', save_best_only=False, mode='max')
    tensbd = TensorBoard(log_dir=log_dir + log_name,
                         batch_size=batch_size,
                         # histogram_freq=100,
                         # embeddings_freq=99,
                         # embeddings_layer_names=embedding_layer_names,
                         # embeddings_data=x_val,
                         # embeddings_metadata=metadata_file,
                         write_images=False)
    class_weights=compute_weight(y_train,range(3))
    print("Class weights %s" % class_weights)

    #### Train ####
    
    history = model.fit(x_train,y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=2,
                        shuffle=True,
                        class_weight=class_weights,
                        callbacks=[modelcheckpnt,
                        log_UAR(x_val, y_val, val_parts),
                        tensbd, csv_logger],
                        validation_data=(x_val,y_val))
    print("History : %s" % history.history)
    loss = history.history['UAR']
    print(loss)
    K.clear_session()
#     return (1.- np.float32(np.max(loss)))
    return {'loss': 1.- np.float32(np.max(loss)), 'status': STATUS_OK}

In [8]:
space = {
    'lr' : 10 ** hp.uniform('lr',-6,-2),
    'num_dense1' : 200 + hp.randint('num_dense1',2000),
    'num_dense2' : 20 + hp.randint('num_dense2',1000)
}

In [ ]:
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            trials=trials,
            max_evals=80)
print('best:', best)

args {'lr': 0.00029060057084501803, 'num_dense1': 436, 'num_dense2': 301}
Class weights {0: 1.9705474705474706, 1: 0.60564430244941425, 2: 1.1885057471264369}
Train on 17061 samples, validate on 5872 samples
Epoch 1/45
5s - loss: 1.3465 - acc: 0.3403 - val_loss: 1.1513 - val_acc: 0.3551
Epoch 2/45
4s - loss: 1.2726 - acc: 0.3682 - val_loss: 1.1614 - val_acc: 0.3484
Epoch 3/45
4s - loss: 1.2385 - acc: 0.3802 - val_loss: 1.1560 - val_acc: 0.3605
Epoch 4/45
4s - loss: 1.2057 - acc: 0.3924 - val_loss: 1.1658 - val_acc: 0.3483
Epoch 5/45
4s - loss: 1.1816 - acc: 0.4020 - val_loss: 1.1738 - val_acc: 0.3409
Epoch 6/45
4s - loss: 1.1565 - acc: 0.4105 - val_loss: 1.1680 - val_acc: 0.3466
Epoch 7/45
4s - loss: 1.1568 - acc: 0.4119 - val_loss: 1.1879 - val_acc: 0.3273
Epoch 8/45
4s - loss: 1.1381 - acc: 0.4198 - val_loss: 1.1869 - val_acc: 0.3319
Epoch 9/45
4s - loss: 1.1239 - acc: 0.4332 - val_loss: 1.1868 - val_acc: 0.3328
Epoch 10/45
4s - loss: 1.1093 - acc: 0.4333 - val_loss: 1.1823 - val_acc

Class weights {0: 1.9705474705474706, 1: 0.60564430244941425, 2: 1.1885057471264369}
Train on 17061 samples, validate on 5872 samples
Epoch 1/45
5s - loss: 1.3155 - acc: 0.3492 - val_loss: 1.1783 - val_acc: 0.3275
Epoch 2/45
4s - loss: 1.1846 - acc: 0.3944 - val_loss: 1.1566 - val_acc: 0.3573
Epoch 3/45
4s - loss: 1.1489 - acc: 0.4166 - val_loss: 1.1745 - val_acc: 0.3334
Epoch 4/45
4s - loss: 1.1108 - acc: 0.4319 - val_loss: 1.1947 - val_acc: 0.3292
Epoch 5/45
4s - loss: 1.0795 - acc: 0.4509 - val_loss: 1.1974 - val_acc: 0.3191
Epoch 6/45
4s - loss: 1.0614 - acc: 0.4564 - val_loss: 1.1730 - val_acc: 0.3404
Epoch 7/45
4s - loss: 1.0462 - acc: 0.4698 - val_loss: 1.1670 - val_acc: 0.3421
Epoch 8/45
4s - loss: 1.0351 - acc: 0.4738 - val_loss: 1.1754 - val_acc: 0.3336
Epoch 9/45
4s - loss: 1.0203 - acc: 0.4815 - val_loss: 1.1728 - val_acc: 0.3333
Epoch 10/45
4s - loss: 1.0075 - acc: 0.4914 - val_loss: 1.1629 - val_acc: 0.3387
Epoch 11/45
4s - loss: 1.0076 - acc: 0.4887 - val_loss: 1.1788 - 

Class weights {0: 1.9705474705474706, 1: 0.60564430244941425, 2: 1.1885057471264369}
Train on 17061 samples, validate on 5872 samples
Epoch 1/45
5s - loss: 1.1661 - acc: 0.4167 - val_loss: 1.1447 - val_acc: 0.3363
Epoch 2/45
4s - loss: 0.9957 - acc: 0.4885 - val_loss: 1.1325 - val_acc: 0.3621
Epoch 3/45
4s - loss: 0.9358 - acc: 0.5300 - val_loss: 1.1959 - val_acc: 0.3365
Epoch 4/45
4s - loss: 0.9007 - acc: 0.5563 - val_loss: 1.1857 - val_acc: 0.3324
Epoch 5/45
4s - loss: 0.8661 - acc: 0.5842 - val_loss: 1.1279 - val_acc: 0.3970
Epoch 6/45
4s - loss: 0.8322 - acc: 0.6039 - val_loss: 1.2298 - val_acc: 0.3336
Epoch 7/45
4s - loss: 0.8006 - acc: 0.6234 - val_loss: 1.2418 - val_acc: 0.3280
Epoch 8/45
4s - loss: 0.7716 - acc: 0.6371 - val_loss: 1.2270 - val_acc: 0.3420
Epoch 9/45
4s - loss: 0.7487 - acc: 0.6517 - val_loss: 1.2141 - val_acc: 0.3520
Epoch 10/45
4s - loss: 0.7233 - acc: 0.6613 - val_loss: 1.2129 - val_acc: 0.3547
Epoch 11/45
4s - loss: 0.6921 - acc: 0.6801 - val_loss: 1.2419 - 

Class weights {0: 1.9705474705474706, 1: 0.60564430244941425, 2: 1.1885057471264369}
Train on 17061 samples, validate on 5872 samples
Epoch 1/45
5s - loss: 1.2423 - acc: 0.3739 - val_loss: 1.1407 - val_acc: 0.3677
Epoch 2/45
4s - loss: 1.1194 - acc: 0.4314 - val_loss: 1.1709 - val_acc: 0.3423
Epoch 3/45
4s - loss: 1.0754 - acc: 0.4609 - val_loss: 1.1721 - val_acc: 0.3442
Epoch 4/45
4s - loss: 1.0349 - acc: 0.4766 - val_loss: 1.1790 - val_acc: 0.3443
Epoch 5/45
4s - loss: 1.0143 - acc: 0.4933 - val_loss: 1.1692 - val_acc: 0.3665
Epoch 6/45
4s - loss: 0.9884 - acc: 0.5075 - val_loss: 1.2060 - val_acc: 0.3287
Epoch 7/45
4s - loss: 0.9656 - acc: 0.5187 - val_loss: 1.2183 - val_acc: 0.3193
Epoch 8/45
4s - loss: 0.9576 - acc: 0.5243 - val_loss: 1.1835 - val_acc: 0.3501
Epoch 9/45
4s - loss: 0.9349 - acc: 0.5402 - val_loss: 1.1678 - val_acc: 0.3701
Epoch 10/45
4s - loss: 0.9257 - acc: 0.5483 - val_loss: 1.1935 - val_acc: 0.3392
Epoch 11/45
4s - loss: 0.9125 - acc: 0.5512 - val_loss: 1.1898 - 

Class weights {0: 1.9705474705474706, 1: 0.60564430244941425, 2: 1.1885057471264369}
Train on 17061 samples, validate on 5872 samples
Epoch 1/45
5s - loss: 1.4087 - acc: 0.3023 - val_loss: 1.1498 - val_acc: 0.3787
Epoch 2/45
4s - loss: 1.3158 - acc: 0.3444 - val_loss: 1.1527 - val_acc: 0.3782
Epoch 3/45
4s - loss: 1.2770 - acc: 0.3582 - val_loss: 1.1592 - val_acc: 0.3753
Epoch 4/45
4s - loss: 1.2370 - acc: 0.3730 - val_loss: 1.1636 - val_acc: 0.3624
Epoch 5/45
4s - loss: 1.2228 - acc: 0.3740 - val_loss: 1.1762 - val_acc: 0.3544
Epoch 6/45
4s - loss: 1.1942 - acc: 0.3872 - val_loss: 1.1746 - val_acc: 0.3552
Epoch 7/45
4s - loss: 1.1747 - acc: 0.3989 - val_loss: 1.1820 - val_acc: 0.3539
Epoch 8/45
6s - loss: 1.1587 - acc: 0.4122 - val_loss: 1.1887 - val_acc: 0.3421
Epoch 9/45
4s - loss: 1.1472 - acc: 0.4157 - val_loss: 1.1914 - val_acc: 0.3394
Epoch 10/45
4s - loss: 1.1395 - acc: 0.4187 - val_loss: 1.1917 - val_acc: 0.3370
Epoch 11/45
4s - loss: 1.1239 - acc: 0.4287 - val_loss: 1.1837 - 

Class weights {0: 1.9705474705474706, 1: 0.60564430244941425, 2: 1.1885057471264369}
Train on 17061 samples, validate on 5872 samples
Epoch 1/45
5s - loss: 1.2334 - acc: 0.3854 - val_loss: 1.1508 - val_acc: 0.3379
Epoch 2/45
4s - loss: 1.1131 - acc: 0.4291 - val_loss: 1.1458 - val_acc: 0.3561
Epoch 3/45
4s - loss: 1.0600 - acc: 0.4640 - val_loss: 1.1964 - val_acc: 0.3263
Epoch 4/45
4s - loss: 1.0294 - acc: 0.4713 - val_loss: 1.1772 - val_acc: 0.3460
Epoch 5/45
4s - loss: 0.9974 - acc: 0.4905 - val_loss: 1.1812 - val_acc: 0.3450
Epoch 6/45
4s - loss: 0.9744 - acc: 0.5109 - val_loss: 1.1899 - val_acc: 0.3351
Epoch 7/45
4s - loss: 0.9653 - acc: 0.5193 - val_loss: 1.1900 - val_acc: 0.3467
Epoch 8/45


In [16]:
trials.trials

[{'book_time': datetime.datetime(2018, 3, 17, 18, 9, 7, 321000),
  'exp_key': None,
  'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'idxs': {'lr': [0], 'num_dense1': [0], 'num_dense2': [0]},
   'tid': 0,
   'vals': {'lr': [-2.6199048020558076],
    'num_dense1': [1332],
    'num_dense2': [260]},
   'workdir': None},
  'owner': None,
  'refresh_time': datetime.datetime(2018, 3, 17, 18, 9, 18, 436000),
  'result': {'loss': 0.6666666567325592, 'status': 'ok'},
  'spec': None,
  'state': 2,
  'tid': 0,
  'version': 0}]